In [11]:
%load_ext autoreload
%autoreload 2
# %cd "/Users/fan/OneDrive\ -\ University\ of\ Southampton/Chandler\'s\ Projects/Edge-Cloud-Resource-Allocation-Using-MARL-and-Auction/scripts"
%cd "../benchmarks"
# %matplotlib inline
# import numpy as np
# import pandas as pd
# import importlib
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# import seaborn as sns
import csv
import pprint
from scipy import stats
import os.path
import pdb
from statistics import mean 
from collections import OrderedDict
import pickle
# from tqdm import tqdm
import json
import sys
sys.path.append('../common')
# import multi_agent_sarsa
# from multi_agent_sarsa import train_multi_|agent_sarsa, execute_multi_agent_sarsa
# import optimal_pricing_m as opm
import offline_optimal_m as oom
# from online_myopic_m import online_myopic
from generate_simulation_data import generate_synthetic_data_edge_cloud
# from other_functions import get_social_welfare
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

stdout = sys.stdout

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/fan/OneDrive - University of Southampton/My-Projects/MARL-Jack/marl-disaster-relief/src/benchmarks


In [2]:
# functions
def set_parameters(seed=0, n_tasks=50, auction_type='first-price'):
    mipgap = 0.1
    n_tasks = int(n_tasks)
    n_time = 20
    seed = seed
    n_nodes = 3
#     resource_coefficient_original = 0.2
    resource_coefficient = 0.5
    high_value_slackness = 0
    low_value_slackness = 0
    valuation_ratio = 20
    resource_ratio = 3
    p_high_value_task = 0.1
    # parameters of fog nodes
    h_r = 50
    l_r = 3
    l_c = 0
    n_c = 2.5
    h_c = 3
    avg_resource_capacity = {0: [h_r, h_r, h_r], 1: [l_r, l_r, l_r], 2: [l_r, l_r, l_r]}
    avg_unit_cost = {0: [h_c, h_c, h_c], 1: [l_c, l_c, l_c], 2: [l_c, l_c, l_c]}
    auction_type = auction_type
    # stop exploration after 5000 steps
    epsilons_tuple = (0.6, 0.3, 0.1)
    epsilon_steps_tuple = (
        int(n_tasks / 4), int(n_tasks / 4),
        int(n_tasks / 4))
    return (seed, mipgap, n_tasks, n_time, n_nodes, resource_coefficient, high_value_slackness, 
        low_value_slackness, valuation_ratio, resource_ratio, p_high_value_task, 
            avg_resource_capacity, avg_unit_cost, epsilons_tuple, epsilon_steps_tuple, auction_type)

## Generate Simulation Data

In [3]:
sys.stdout = stdout
seed = 127
# set the parameters
(seed, mipgap, n_tasks, n_time, n_nodes, resource_coefficient, high_value_slackness, 
low_value_slackness, valuation_ratio, resource_ratio, p_high_value_task, avg_resource_capacity, 
 avg_unit_cost, epsilons_tuple, epsilon_steps_tuple, auction_type) = set_parameters(seed=seed)
# generate synthetic data for the simulation
df_tasks, df_nodes, n_time, n_tasks, n_nodes = \
    generate_synthetic_data_edge_cloud(avg_resource_capacity, avg_unit_cost, n_tasks=n_tasks,
                                       n_time=n_time, n_nodes=n_nodes,
                                       p_high_value_tasks=p_high_value_task,
                                       high_value_slackness_lower_limit=high_value_slackness,
                                       high_value_slackness_upper_limit=high_value_slackness,
                                       low_value_slackness_lower_limit=low_value_slackness,
                                       low_value_slackness_upper_limit=low_value_slackness,
                                       k_resource=resource_coefficient,
                                       resource_demand_high=resource_ratio,
                                       vc_ratio=valuation_ratio,
                                       seed=seed)
df_tasks = df_tasks.rename(columns={"storage": "DISK"})
df_nodes = df_nodes.rename(
    columns={"storage": "DISK", "storage_cost": "DISK_cost"})

print("df_tasks:")
print(df_tasks.head(30))
print("df_nodes:")
print(df_nodes)
# # print the total value of tasks
# total_value = 0
# for i in range(n_tasks):
#     total_value += (df_tasks.loc[i, "valuation_coefficient"] *
#                     df_tasks.loc[i, "usage_time"])
# print(f"n_tasks={n_tasks}")
# print(f"num of time steps={n_time}")
# print(f"total value of tasks = {total_value}")

df_tasks:
    valuation_coefficient  arrive_time  start_time  deadline  usage_time  \
0               73.671596     0.598075           1         4           4   
1               87.815222     0.679371           1         4           4   
2               71.553045     1.166257           2         5           4   
3               80.053042     1.213587           2         5           4   
4               54.988069     1.466707           2         5           4   
5               95.585434     1.511640           2         5           4   
6               62.577778     1.601125           2         5           4   
7               55.515982     1.783614           2         5           4   
8               78.397661     1.954847           2         5           4   
9               96.076191     2.167929           3         6           4   
10              91.219668     2.756314           3         6           4   
11            1094.449565     2.970436           3         6           4   
12

### Offline Optimal

In [4]:
# run Offline Optimal algo.
mipgap=0.1
social_welfare, social_welfare_solution, number_of_allocated_tasks, allocation_scheme = \
    oom.offline_optimal(df_tasks, df_nodes, n_time, n_tasks, n_nodes,
                        mipgap=mipgap)
print("social welfare:", social_welfare)
print("number of allocated tasks:", number_of_allocated_tasks)

100%|██████████| 50/50 [00:00<00:00, 24678.18it/s]


social welfare: 35308.299363929335
number of allocated tasks: 50


In [17]:
# which time step is vacant for node 1
print("vacant time steps for node 1")
for i in range(len(next(iter(allocation_scheme['node_1'].values())))):
    vacant = True
    for k in allocation_scheme['node_1'].values():
        if k[i] != '..':
            vacant = False
    if vacant == True:
        print(i)

vacant time steps for node 1
0


In [18]:
# which time step is vacant for node 2
print("vacant time steps for node 2")
for i in range(len(next(iter(allocation_scheme['node_2'].values())))):
    vacant = True
    for k in allocation_scheme['node_2'].values():
        if k[i] != '..':
            vacant = False
    if vacant == True:
        print(i)

vacant time steps for node 2
0


## Get average social welfare of Offline Optimal

In [12]:
total_sw = 0
n_trials = 500
for seed in range(0, n_trials):
    print(f"seed = {seed}")
    # set the parameters
    (seed, mipgap, n_tasks, n_time, n_nodes, resource_coefficient, high_value_slackness, 
    low_value_slackness, valuation_ratio, resource_ratio, p_high_value_task, avg_resource_capacity, 
     avg_unit_cost, epsilons_tuple, epsilon_steps_tuple, auction_type) = set_parameters(seed=seed)
    # generate synthetic data for the simulation
    df_tasks, df_nodes, n_time, n_tasks, n_nodes = \
        generate_synthetic_data_edge_cloud(avg_resource_capacity, avg_unit_cost, n_tasks=n_tasks,
                                           n_time=n_time, n_nodes=n_nodes,
                                           p_high_value_tasks=p_high_value_task,
                                           high_value_slackness_lower_limit=high_value_slackness,
                                           high_value_slackness_upper_limit=high_value_slackness,
                                           low_value_slackness_lower_limit=low_value_slackness,
                                           low_value_slackness_upper_limit=low_value_slackness,
                                           k_resource=resource_coefficient,
                                           resource_demand_high=resource_ratio,
                                           vc_ratio=valuation_ratio,
                                           seed=seed)
    df_tasks = df_tasks.rename(columns={"storage": "DISK"})
    df_nodes = df_nodes.rename(
        columns={"storage": "DISK", "storage_cost": "DISK_cost"})
    # run Offline Optimal algo.
    mipgap=0.1
    social_welfare, social_welfare_solution, number_of_allocated_tasks, allocation_scheme = \
        oom.offline_optimal(df_tasks, df_nodes, n_time, n_tasks, n_nodes,
                            mipgap=mipgap)
    print("social welfare:", social_welfare)
    print("number of allocated tasks:", number_of_allocated_tasks)
    total_sw += social_welfare
    
avg_sw = total_sw/n_trials
print(f"average social welfare = {avg_sw}")

seed = 0


100%|██████████| 50/50 [00:00<00:00, 16430.21it/s]


social welfare: 36233.559396752724
number of allocated tasks: 50
seed = 1


100%|██████████| 50/50 [00:00<00:00, 21384.24it/s]


social welfare: 37140.361643183816
number of allocated tasks: 50
seed = 2


...

social welfare: 36822.6011447913
number of allocated tasks: 50
seed = 498


100%|██████████| 50/50 [00:00<00:00, 18096.06it/s]


social welfare: 38264.22102487491
number of allocated tasks: 50
seed = 499


100%|██████████| 50/50 [00:00<00:00, 20998.82it/s]


social welfare: 42252.24831573321
number of allocated tasks: 50
average social welfare = 38814.241600511305


In [8]:
1/0

ZeroDivisionError: division by zero

### Online Myopic:

In [ ]:
# run online myopic algo.
social_welfare, number_of_allocated_tasks, allocation_scheme = \
    online_myopic(df_tasks, df_nodes, n_time, n_tasks, n_nodes)
print("social welfare:", social_welfare)
print("number of allocated tasks:", number_of_allocated_tasks)

In [ ]:
raise SystemExit("Stop✋")

In [ ]:
# social_welfare_partial = get_social_welfare(df_tasks, df_nodes,
#                                             allocation_scheme, start_id=int(n_tasks/3),
#                                             end_id=int(n_tasks/3*2))
# social_welfare_partial

### Optimal Pricing:

In [ ]:
# run Optimal Pricing algo.
# set price range
column = df_tasks['valuation_coefficient']
price_upper_value = column.max()
price_lower_value = 0
# granularity = (price_upper_value - price_lower_value) / 5
granularity = (price_upper_value - price_lower_value) / 10

# run OP algo.
list_social_welfare, list_number_of_allocated_tasks, optimal_phi, allocation_scheme = \
    opm.optimal_pricing(df_tasks, df_nodes, n_time, n_tasks, n_nodes,
                        granularity=granularity,
                        price_upper_value=price_upper_value,
                        price_lower_value=price_lower_value, global_phi=False)
print("social welfare:", list_social_welfare)
print("number of allocated tasks:", list_number_of_allocated_tasks)
print(f"optimal_phi = {optimal_phi}")

# # the social welfare of 100 tasks in the middle
# social_welfare_partial = get_social_welfare(df_tasks, df_nodes,
#                                             allocation_scheme, start_id=99,
#                                             end_id=199)
# print(f"social_welfare_partial={social_welfare_partial}")

In [ ]:
raise SystemExit("Stop Here✋")

### Reverse Auction V1

### Execution

In [ ]:
number_of_tasks = 300
num_actions = 10
valuation_coefficient_ratio = 2
high_value_proportion = 0.1
time_length = int(number_of_tasks / 10)
num_trials = seed + 1
resource_ratio = 1
seed=0
filehandler = open(f"./trained_agents/reverse_auction_v1_seed={seed}_rc={resource_coefficient_original}_agents", 'rb')
agents_list = pickle.load(filehandler)
sw_list, total_value, df_tasks_2, df_nodes, agents_list, allocation_scheme = \
    execute_multi_agent_sarsa(num_actions=num_actions,
                              time_length=time_length,
                              high_value_proportion=high_value_proportion,
                              total_number_of_steps=number_of_tasks,
                              num_fog_nodes=6,
                              resource_coefficient_original=resource_coefficient_original,
                              valuation_coefficient_ratio=valuation_coefficient_ratio,
                              number_of_runs=num_trials, plot_bool=True,
                              bool_decay=True,
                              resource_ratio=resource_ratio,
                              agents_list=agents_list, training_seed=seed,
                              verbose=False)

sys.stdout = stdout
print(f"total value of tasks = {total_value}")
social_welfare = sw_list[-1]
print(f"total social welfare = {social_welfare}")
social_welfare_partial = sw_list[199] - sw_list[99]
print(f"social_welfare_partial={social_welfare_partial}")
print(df_tasks_2.head())

In [ ]:
raise SystemExit("Stop✋")

## Training

In [ ]:
# set the parameters
(seed, mipgap, n_tasks, n_time, n_nodes, resource_coefficient, high_value_slackness, 
low_value_slackness, valuation_ratio, resource_ratio, p_high_value_task, avg_resource_capacity,
 avg_unit_cost, epsilons_tuple, epsilon_steps_tuple, auction_type) = set_parameters(30000)
num_actions=10
verbose=False

# train the agents
(sw_list, total_value, df_tasks_1, df_nodes, agents_list,
allocation_scheme) = train_multi_agent_sarsa(avg_resource_capacity, avg_unit_cost, 
                                        alpha=0.02,
            beta=0.001, epsilon_tuple=epsilons_tuple, epsilon_steps_tuple=epsilon_steps_tuple,
            high_value_proportion=p_high_value_task, num_actions=num_actions,
            time_length=n_time, total_number_of_steps=n_tasks, num_fog_nodes=6,
            valuation_coefficient_ratio=valuation_ratio,
            resource_ratio=resource_ratio, seed=seed, verbose=verbose,
            plot_bool=True, auction_type=auction_type)

# save the trained agents
filehandler = open(f"../trained_agents/reverse_auction_v1_seed={seed}_agents",
                   'wb')
pickle.dump(agents_list, filehandler)

# print the results
print("social welfare:", sw_list[-1])
print("total value:", total_value)

### Execution

In [ ]:
# get the trained agents
filehandler = open(f"../trained_agents/reverse_auction_v1_seed={seed}_agents", 'rb')
agents_list = pickle.load(filehandler)

# set the parameters
(seed, mipgap, n_tasks, n_time, n_nodes, resource_coefficient, high_value_slackness, 
low_value_slackness, valuation_ratio, resource_ratio, p_high_value_task, avg_resource_capacity,
 avg_unit_cost, epsilons_tuple, epsilon_steps_tuple, auction_type) = set_parameters(10000)
num_trials=1

# allocate tasks:
sw_list, total_value, df_tasks_2, df_nodes, agents_list, allocation_scheme = \
    execute_multi_agent_sarsa(avg_resource_capacity, avg_unit_cost, num_actions=num_actions,
                              time_length=n_time,
                              high_value_proportion=p_high_value_task,
                              total_number_of_steps=n_tasks,
                              num_fog_nodes=6,
                              valuation_coefficient_ratio=valuation_ratio,
                              number_of_runs=num_trials, plot_bool=True,
                              bool_decay=True,
                              resource_ratio=resource_ratio,
                              agents_list=agents_list, training_seed=seed,
                              verbose=False, auction_type=auction_type)

sys.stdout = stdout
print(f"auction type = {auction_type}")
print(f"total value of tasks = {total_value}")
social_welfare = sw_list[-1]
print(f"total social welfare = {social_welfare}")
# social_welfare_partial = sw_list[int(number_of_tasks/3*2)] - sw_list[int(number_of_tasks/3)]
# print(f"social_welfare_partial={social_welfare_partial}")
# print(df_tasks_2.head())

In [ ]:
raise SystemExit("Stop✋")

In [ ]:
# make sure the tasks are the same.
df_tasks_2.head()
df_tasks.head()
df_tasks_2.tail()
df_tasks.tail()

In [4]:
# test normalisation
df = df_nodes
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df
task_info = normalized_df.iloc[0]
task_info = task_info.to_numpy()
task_info

,CPU,RAM,DISK,CPU_cost,RAM_cost,DISK_cost
0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0


array([1., 1., 1., 1., 1., 1.])

In [5]:
future_occup = np.zeros((6, 10, 3), dtype=float)
future_occup = future_occup.flatten()
future_occup

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
df_nodes
for id, info in df_nodes.iterrows():
    print(id)
    print(info.get('CPU'))

,CPU,RAM,DISK,CPU_cost,RAM_cost,DISK_cost
0,50.0,50.0,50.0,3.0,3.0,3.0
1,3.0,3.0,3.0,0.0,0.0,0.0
2,3.0,3.0,3.0,0.0,0.0,0.0


0
50.0
1
3.0
2
3.0
